# Import

In [1]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml


# URL Rule

In [2]:
# comp_id = 380725    # 기업 고유 UID
p_num = 1      # 리뷰 페이지 (4 reviews / page)

### query로 CompID 가져오기

In [3]:
Keyword = '삼성전자'

In [4]:
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(Keyword))
# print(search_req.text)

In [5]:
search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text

In [6]:
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

[<a href="/Comp/CompSummary/380725">
                 삼성전자
             </a>]

In [7]:
comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
comp_id

'380725'

In [8]:
URL = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=5000&isNew=false&employType=1&isEmploy=false&jobCode='
print(URL)

https://www.catch.co.kr/api/v1.0/comp/reviewInfo/380725/commentList?currentPage=1&pageSize=5000&isNew=false&employType=1&isEmploy=false&jobCode=


# Json to DataFrame 

In [9]:
# Json 형식의 웹 데이터 가져오기
raw_data = requests.get(URL).json()
# print(raw_data)

In [10]:
data = raw_data['reviewList']
data[0]

{'idx': 209151,
 'CompID': '380725',
 'CompName': '삼성전자',
 'RegDate': '2023-08-10T08:41:40.253Z',
 'CI': '380725.jpg',
 'EmployText': '현직',
 'Gender2': 2,
 'RecomName': '추천',
 'EmployType': '정규직',
 'NewOld': '신입',
 'Answer': None,
 'Good': '어학  지원\r\n칼퇴  가능\r\n자울 출퇴근제\r\n식사 제공\r\n통근버스',
 'Bad': '수직적 조직문화\r\n선택적 복리후생\r\n불투명한 소통\r\n기타등등',
 'UsefulY': 0,
 'MyUsefulY': None,
 'MyOpinion': None,
 'CareerYear': 13,
 'CareerYearYN': 'Y',
 'Keyword1': None,
 'Keyword2': None,
 'Keyword3': None,
 'Keyword1YN': 'N',
 'Keyword2YN': 'N',
 'Keyword3YN': 'N',
 'JobName': '생산관리/공정관리/품질관리',
 'Area': '경북',
 'MyStarScore': 4}

In [11]:
df = pd.DataFrame(data)
df.head()

,idx,CompID,CompName,RegDate,CI,EmployText,Gender2,RecomName,EmployType,NewOld,...,CareerYearYN,Keyword1,Keyword2,Keyword3,Keyword1YN,Keyword2YN,Keyword3YN,JobName,Area,MyStarScore
0,209151,380725,삼성전자,2023-08-10T08:41:40.253Z,380725.jpg,현직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,생산관리/공정관리/품질관리,경북,4.0
1,209113,380725,삼성전자,2023-08-09T03:31:28.130Z,380725.jpg,전직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,생산/제조/설비/조립,경기,3.0
2,208104,380725,삼성전자,2023-06-26T05:25:22.170Z,380725.jpg,전직,2,추천,정규직,신입,...,N,None,None,None,N,N,N,웹개발,서울,4.6
3,208095,380725,삼성전자,2023-06-26T00:15:57.717Z,380725.jpg,현직,2,추천,정규직,신입,...,Y,None,None,None,N,N,N,일반영업,서울,2.6
4,207978,380725,삼성전자,2023-06-21T05:23:01.440Z,380725.jpg,전직,2,비추,정규직,경력,...,Y,None,None,None,N,N,N,기획/전략/경영,경기,2.4


In [12]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'CI', 'EmployText', 'Gender2',
       'RecomName', 'EmployType', 'NewOld', 'Answer', 'Good', 'Bad', 'UsefulY',
       'MyUsefulY', 'MyOpinion', 'CareerYear', 'CareerYearYN', 'Keyword1',
       'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN',
       'JobName', 'Area', 'MyStarScore'],
      dtype='object')

In [13]:
df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 'CareerYearYN',
            'MyUsefulY', 'RecomName', 'MyOpinion', 'CareerYear', 'Keyword1', 
            'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
            axis=1, inplace=True)

df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'Good', 'Bad',
       'JobName', 'MyStarScore'],
      dtype='object')

In [14]:
# 정규식으로 이스케이프 문자 제거        # Good, Bad 리뷰 항목에만 적용.
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df

,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,2023-08-10T08:41:40.253Z,현직,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,2023-08-09T03:31:28.130Z,전직,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,2023-06-26T05:25:22.170Z,전직,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,2023-06-26T00:15:57.717Z,현직,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,2023-06-21T05:23:01.440Z,전직,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,2020-09-26T11:23:26.790Z,전직,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,2020-09-24T05:02:07.343Z,현직,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,2020-09-12T07:34:44.560Z,전직,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,2020-09-07T12:49:14.490Z,전직,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


In [15]:
# RegDate 컬럼의 데이터 - 문자열 9번째 까지 출력 (YYYYMMDD 형식의 날짜 표기)
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())
df

,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,20230810,현직,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,20230809,전직,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,20230626,전직,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,20230626,현직,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,20230621,전직,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,20200926,전직,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,20200924,현직,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,20200912,전직,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,20200907,전직,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


# For 문으로 만들어보기
    - 리뷰 데이터가 있는 페이지까지 반복

In [16]:
rv = []

for p_num in range(1, 52) :
    url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=4&isNew=false&employType=1&isEmploy=false&jobCode='
    rv.append(requests.get(url).json()['reviewList'])


data = list(itertools.chain.from_iterable(rv))
df = pd.DataFrame(data)


df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 
            'RecomName', 'CareerYearYN', 'MyUsefulY', 'MyOpinion', 'Area',
            'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN'], 
            axis=1, inplace=True)
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:6].strip())


df

,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,202308,현직,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,202308,전직,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,202306,전직,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,202306,현직,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,202306,전직,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...
199,148439,380725,삼성전자,202110,전직,돈을 많이 주고 이직에 최소한 경력으로 도움이 된다 복지는 좋다,돈을 많이 주는만큼 일을 많이 시키고 유연근무제도 안유연하고 야근이 많다,반도체/디스플레이,2.4
200,148244,380725,삼성전자,202110,현직,"자울출퇴근제이고, 합의된 비율로 재택근무가능하다. 예전에 비해 조직 분위기가 강압적...",제품이 너무 많고 일정이 타이트하여 개발자들과 관련부서(검증 등)에 로드가 심하다....,전기/전자/제어,4.0
201,147914,380725,삼성전자,202110,현직,눈치 볼 필요없이 칼퇴가 가능하다. 연차를 쓸 수 있다.,가끔 업무강도가 너무 높고 야근이 갑자기 생겨서 불편하다.,사무/총무/법무,4.6
202,145509,380725,삼성전자,202106,전직,업무문화가 계속해서 변화를 시도해서 좋다. 뭔가 비젼을 제시하고 있어서 좋다,정년을 보장받기는 어렵다. 정리해고 할때는 가차없이 내보내고 노조가 없어서 아무소리...,전기/전자/제어,3.0


# While 문으로 만들기 
    - 그 다음 페이지에 리뷰 데이터 개수가 0 이면 멈춤


In [17]:
# len(requests.get(url).json()['reviewList'])

In [18]:
# Catch 사이트에 검색을 하면, 최상위 검색결과의 comp_ui를 받아오기

comp_name = '삼성전자'
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(comp_name)) # Keyword = comp_name
# print(search_req.text)

search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
comp_id

'380725'

In [19]:
# 위에서 받아온 comp_id를 가지고 데이터 받아와서 데이터프레임으로 만들기

try : 
    # comp_id = 380725
    p_num = 1

    rv = []


    while True : 

        url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=2000&isNew=false&employType=1&isEmploy=false&jobCode='
        rv.append(requests.get(url).json()['reviewList'])

        p_num += 1

        if len(requests.get(url).json()['reviewList']) == 0 :
            break


    data = list(itertools.chain.from_iterable(rv))      # = 리스트 컴프리헨션
    df = pd.DataFrame(data)


    df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 
            'RecomName', 'CareerYearYN', 'MyUsefulY', 'MyOpinion', 'Area',
            'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN'], 
            axis=1, inplace=True)
            
    df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:6].strip())
    


except :
    print("리뷰가 존재하지 않습니다.")


df


,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
0,209151,380725,삼성전자,202308,현직,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0
1,209113,380725,삼성전자,202308,전직,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3.0
2,208104,380725,삼성전자,202306,전직,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,4.6
3,208095,380725,삼성전자,202306,현직,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,2.6
4,207978,380725,삼성전자,202306,전직,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2.4
...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,202009,전직,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4.2
302,120816,380725,삼성전자,202009,현직,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4.0
303,120504,380725,삼성전자,202009,전직,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4.0
304,120412,380725,삼성전자,202009,전직,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3.4


# 컬럼명 및 데이터 변경

### 컬럼명 변경

In [20]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'Good', 'Bad',
       'JobName', 'MyStarScore'],
      dtype='object')

In [21]:
df.rename(columns=
            {'idx':'review_uid', 
            'CompID':'comp_uid',
            'CompName':'comp_name', 
            'RegDate':'review_date', 
            'EmployText':'is_office', 
            'Good':'review_pos', 
            'Bad':'review_neg', 
            'MyStarScore':'review_rate', 
            'JobName':'position'}, 
            inplace=True)

df.head(1)

,review_uid,comp_uid,comp_name,review_date,is_office,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,202308,현직,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4.0


### 전/현직 여부를 Boolean 값으로 변경

In [22]:
df[['is_office']]

,is_office
0,현직
1,전직
2,전직
3,현직
4,전직
...,...
301,전직
302,현직
303,전직
304,전직


In [23]:
df['is_office'] = df['is_office'].apply(lambda x: x.replace('현직', '1').strip() == '1')
df[['is_office']]

,is_office
0,True
1,False
2,False
3,True
4,False
...,...
301,False
302,True
303,False
304,False


### review_rate 의 값들을 전부 정수형으로 반올림

In [24]:
df['review_rate'].round(0).astype(int)

0      4
1      3
2      5
3      3
4      2
      ..
301    4
302    4
303    4
304    3
305    3
Name: review_rate, Length: 306, dtype: int32

In [25]:
df['review_rate'] = df['review_rate'].round(0).astype(int)
df.head()

,review_uid,comp_uid,comp_name,review_date,is_office,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,202308,True,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4
1,209113,380725,삼성전자,202308,False,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3
2,208104,380725,삼성전자,202306,False,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,5
3,208095,380725,삼성전자,202306,True,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,3
4,207978,380725,삼성전자,202306,False,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2


### 긍/부정 리뷰들을 한 컬럼에 합치기

In [26]:
a = df.drop(['review_pos'], axis=1).rename(columns = {'review_neg':'review_cont'})
b = df.drop(['review_neg'], axis=1).rename(columns = {'review_pos':'review_cont'})

new_df = pd.concat([a, b])

new_df

,review_uid,comp_uid,comp_name,review_date,is_office,review_cont,position,review_rate
0,209151,380725,삼성전자,202308,True,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4
1,209113,380725,삼성전자,202308,False,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3
2,208104,380725,삼성전자,202306,False,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,5
3,208095,380725,삼성전자,202306,True,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,3
4,207978,380725,삼성전자,202306,False,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2
...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,202009,False,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,판매/캐셔/매장관리,4
302,120816,380725,삼성전자,202009,True,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,기획/전략/경영,4
303,120504,380725,삼성전자,202009,False,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,재무/세무/IR,4
304,120412,380725,삼성전자,202009,False,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,네트워크/서버/보안,3


# DataFrame to CSV file

In [27]:
SAVE_PATH = r'./data/'

In [28]:
df['comp_name'][0]

'삼성전자'

In [29]:
comp = df['comp_name'][0]

In [30]:
# 디렉토리 생성 (이미 존재하면 생성하지 않음)

os.makedirs(SAVE_PATH, exist_ok=True)

# csv 파일로 저장.

file_name = f"{comp}_catch.csv"
save_file_path = os.path.join(SAVE_PATH, file_name)

df.to_csv(save_file_path, index=False, encoding = "utf-8")

In [31]:
pd.read_csv(save_file_path)

,review_uid,comp_uid,comp_name,review_date,is_office,review_pos,review_neg,position,review_rate
0,209151,380725,삼성전자,202308,True,어학 지원 칼퇴 가능 자울 출퇴근제 식사 제공 통근버스,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4
1,209113,380725,삼성전자,202308,False,눈치 볼 필요없이 칼퇴가 가능하다. 초봉이 상대적으로 높은 편이다.,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3
2,208104,380725,삼성전자,202306,False,정치질을 잘 한다면 일을 열심히 할 필요가 없는 회사. 밥 잘나오고 ot 잘 챙겨준...,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,5
3,208095,380725,삼성전자,202306,True,눈치 안보고 연차 사용가능 복지가 매우 좋다 헬스장 샤워실 등등,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,3
4,207978,380725,삼성전자,202306,False,복지 면에서는 타의 추종 불허. 연봉은 성과급 변수 있지만 대체로 만족,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2
...,...,...,...,...,...,...,...,...,...
301,120892,380725,삼성전자,202009,False,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,단점은 없다. 그냥 세계 최고의 회사답게 정말 좋다. 사람들이 친절하다.,판매/캐셔/매장관리,4
302,120816,380725,삼성전자,202009,True,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,관리가 심하고 보고문화가 지나치게 심하다..,기획/전략/경영,4
303,120504,380725,삼성전자,202009,False,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,일이 빡시고 성과주의라 스트레스는 어쩔 수 없는듯,재무/세무/IR,4
304,120412,380725,삼성전자,202009,False,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,"부서별 편차가 너무 크다. 임원들의 단기 성과가 중요해서, 직원들의 장기적인 성장을...",네트워크/서버/보안,3


# 함수화

In [32]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml

In [33]:
SAVE_PATH = r'./data/'
# SAVE_PATH = r'/app/data/reviews/'     <--- for Docker

In [34]:
re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))

'[<ahref="/Comp/CompSummary/380725">삼성전자</a>]'

In [35]:
re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]

'삼성전자'

In [36]:
def get_review(comp = str): 
    
    search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
    search_req = requests.get(search_url.format(comp)) # Keyword = comp_name
    # print(search_req.text)

    search_soup = BeautifulSoup(search_req.text, 'lxml')
    # search_soup.text
    search_soup.select('li:nth-child(1) > div.txt > p.name > a')

    comp_id = re.sub(r'[^0-9]', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))
    comp_name = re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]




    p_num = 1

    rv = []



    try : 

        while True : 

            url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=5000&isNew=false&employType=1&isEmploy=false&jobCode='
            rv.append(requests.get(url).json()['reviewList'])

            p_num += 1

            if len(requests.get(url).json()['reviewList']) == 0 :
                break


        data = list(itertools.chain.from_iterable(rv))
        df = pd.DataFrame(data)


        df.drop(['idx', 'CompID', 'CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 
                'RecomName', 'CareerYearYN', 'MyUsefulY', 'MyOpinion', 'Area', 'CareerYear',
                'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN'], 
                axis=1, inplace=True)
        

        df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
        df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())

        df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:6].strip())

        df['EmployText'] = df['EmployText'].apply(lambda x: x.replace('현직', '1').strip() == '1')



        df.rename(columns=
           {'RegDate':'review_date', 
           'EmployText':'is_office', 
           'Good':'review_pos', 
           'Bad':'review_neg', 
           'MyStarScore':'review_rate', 
           'JobName':'position'
           }, inplace=True)


        df['review_rate'] = df['review_rate'].round(0).astype(int)



        # 긍/부정 리뷰들만 찢어서 합치기

        a = df.drop(['review_pos'], axis=1).rename(columns = {'review_neg':'review_cont'})
        b = df.drop(['review_neg'], axis=1).rename(columns = {'review_pos':'review_cont'})

        new_df = pd.concat([a, b])

        # return new_df




        # csv 파일로 저장.

        os.makedirs(SAVE_PATH, exist_ok=True)

        file_name = f"{comp_name}_catch.csv"
        save_file_path = os.path.join(SAVE_PATH, file_name)

        new_df.to_csv(save_file_path, index=False, encoding = "utf-8")
        
        return pd.read_csv(save_file_path)


        
    except : 
        print("리뷰가 존재하지 않습니다.")




In [37]:
get_review('SK하이')

,CompName,review_date,is_office,review_cont,position,review_rate
0,SK하이닉스,202307,False,업무량이 꽤 있는 편이라고 생각합니다. 워라밸 측면에서는 좀 아쉬움,기획/전략/경영,3
1,SK하이닉스,202306,True,1. 반도체 업계 특성상 사이클을 탐 2. 팀바팀이지만 야근하는 경우도 많음 3. ...,반도체/디스플레이,5
2,SK하이닉스,202306,True,다만 제조기술이 아닌팀 그리고 일부 팀들은 야근이 매우 많은팀들이 있긴하다. 추가로...,생산/제조/설비/조립,4
3,SK하이닉스,202306,True,보수적인 상사분들이 있다. (보통 나이가 많으신 분들 중에) 유연근무제이긴 하나 부...,생산/제조/설비/조립,4
4,SK하이닉스,202306,True,서울 경기권에서 지역 마다 다르지만 출퇴근 시간이 오래걸린다. 조직 문화가 다소 경...,생산관리/공정관리/품질관리,4
...,...,...,...,...,...,...
167,SK하이닉스,202011,True,"대기업인만큼 복지가 잘 되어있음, 하루 네끼 무료",반도체/디스플레이,4
168,SK하이닉스,202011,True,반도체가 호황이라 돈을 많이 주지만 일하는 만큼 받는건지는 모르겠음,반도체/디스플레이,4
169,SK하이닉스,202011,False,복지 좋음. 식사 시설 등 제공 좋습니다. 만족해요 다양한 선택권,반도체/디스플레이,4
170,SK하이닉스,202010,True,복지혜택이 많다. 월급이 많다. 성과급이 많다,반도체/디스플레이,5


In [38]:
get_review('화인석재')

리뷰가 존재하지 않습니다.


In [39]:
get_review('삼성전자')

,CompName,review_date,is_office,review_cont,position,review_rate
0,삼성전자,202308,True,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4
1,삼성전자,202308,False,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3
2,삼성전자,202306,False,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,5
3,삼성전자,202306,True,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,3
4,삼성전자,202306,False,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2
...,...,...,...,...,...,...
607,삼성전자,202009,False,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,판매/캐셔/매장관리,4
608,삼성전자,202009,True,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,기획/전략/경영,4
609,삼성전자,202009,False,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,재무/세무/IR,4
610,삼성전자,202009,False,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,네트워크/서버/보안,3


In [40]:
get_review('네이버')

,CompName,review_date,is_office,review_cont,position,review_rate
0,네이버,202306,False,다른 테크 회사들 대비해서는 조금 더 경직된 구조이긴 함. 아무래도 대기업이니,기획/전략/경영,4
1,네이버,202306,True,"가끔 업무강도가 높음, 재택의 영향으로 직원들 얼굴 볼 기회가 적음, 성과 내기가 어려움",웹개발,5
2,네이버,202305,False,업무량이 많아서 주말에 일을 할 경우가 있다 그런 거 말고는 딱히 없다,B2B영업/기술영업,4
3,네이버,202305,False,"젊꼰이 많다 높은 업무강도 책임감등이 필요하다,,,,,",광고/시각디자인,3
4,네이버,202305,True,업무강도가 높고 항상 일이 많다. 다들 열심히 해서 나도 잘해야할 것 같은 압박이 있다.,기획/전략/경영,4
...,...,...,...,...,...,...
81,네이버,202011,True,"복지가 최고다. 회사 네임벨류 덕분에 스스로 자부심을 느낄 수 있다. 또한, 팀원들...",마케팅/PR/분석,4
82,네이버,202011,False,업계 최고 기업 중 하나라서 이직 시에 유리하다.,콘텐츠/사이트운영,5
83,네이버,202011,True,"네이버는 좋은 회사입니다, 회사 기업문화도 좋습니다",디자인기타,5
84,네이버,202011,False,사내복지와 분위기 등 직원 복지부분은 준비가 매우 잘 되어있음,네트워크/서버/보안,4


In [41]:
get_review('카카오')

,CompName,review_date,is_office,review_cont,position,review_rate
0,카카오,202307,True,경영진의 문제가 심각 일단 사이즈는 큰데 앞으로 뭐먹고..?,웹기획/PM/웹마케팅,4
1,카카오,202306,True,이쪽 특이긴 하지만 연봉 인상률이 낮음 자유로운 분위기가 독이 되는게 본인이 자율적...,응용프로그램개발,4
2,카카오,202306,True,진입장벽이 있고 업계 전반적으로 성장가능성이 높음. 연봉인상률 양호함. 부서에 따라...,빅데이터/AI,5
3,카카오,202305,False,업무강도가 있는편이고 개인적인 업무 편향으로 먼가 혼자 일한다,마케팅/PR/분석,5
4,카카오,202305,True,업무에 체계가 없어서 지시도 명확하지 않고..내가 무슨일을 하고 있는거지 하는 확신...,인사/노무/교육,4
...,...,...,...,...,...,...
83,카카오,202011,False,자유로운 분위기입니다. 복장자유라서 편하게 다닐 수 있습니다,마케팅/PR/분석,4
84,카카오,202011,True,"수평적이고 자기주도적인 업무 문화, 자유로운 분위기, 출퇴근 자유",사무/총무/법무,3
85,카카오,202011,True,대기업이라 만족하고 좋아요 안정적이네요ㅎㅎ,통신기술/네트워크구축,4
86,카카오,202011,True,복지시스템이 엄청 잘되있어서 자존감이 높아져요. 복장자유가 너무 좋아요,마케팅/PR/분석,5


In [42]:
from new_catch import get_review

In [43]:
get_review('NC소프트')

,CompName,review_date,is_office,review_cont,position,review_rate
0,엔씨소프트,202306,False,오락가락과 리셋을 몇년째 반복하는 프로젝트. 지시 수행에 목메는 리더들. 비인간적인...,게임,3
1,엔씨소프트,202305,True,업무공간 간 복지차이가 조금 있음 지나친 추첨제도로 약간 피곤..,기획/전략/경영,4
2,엔씨소프트,202305,True,노사관계가 원만하지 못하며 임원진과 직원과의 차별이 심합니다 대기업축에 속하지만 가...,게임,4
3,엔씨소프트,202210,True,정치질이 상당해서 잘못걸린 경우 빠른 퇴사가 정답인 경우가 많다,응용프로그램개발,5
4,엔씨소프트,202205,False,인센티브가 높은 대신 기본 연봉은 별로. 탑다운에 보여주기식 업무가 심하다. 라인 ...,웹기획/PM/웹마케팅,2
5,엔씨소프트,202201,True,개발자나 게임 기획 아니면 별로 인센이 없음 너무 조직이 세분화 되어 있어 장점이자...,웹기획/PM/웹마케팅,3
6,엔씨소프트,202201,True,개발 게임쪽 아니면 비추 팀바팀 상격이 강함 좋으면서도…. 잘 모르겠음,웹기획/PM/웹마케팅,4
7,엔씨소프트,202110,False,군대같은 분위기. 상명하복 및 윗사람의 말을 무조건적으로 잘 들어야 회사안에서 클 ...,웹기획/PM/웹마케팅,2
8,엔씨소프트,202110,False,사회 평판에 너무 많은 분위기를 타다 보니 눈칫밥이 제로 면 회사 생활이 조금은 불...,게임,4
9,엔씨소프트,202108,True,최근 연봉이 오른 이후로 보너스가 얼마나 나올지 모름. 연봉상승률 짬,응용프로그램개발,4
